Import the dataset provided into a Pandas dataframe and print out the skewness of each numeric feature

In [3]:
# Question 1:
import pandas as pd
df = pd.read_csv('student_enrollment_clean.csv')
skewness = df.skew(numeric_only=True)

# Print the skewness
print("\nSkewness of numeric features:")
print(skewness)




Skewness of numeric features:
ENROLL_COUNT             0.000000
NUMBER_AVERAGE           1.616012
HOURS_ATTEMPTED          3.272376
HOURS_EARNED             3.457838
AR_BALANCE_AMOUNT        4.232079
DAYS_ABSENT              0.000000
REENTRY_NUMBER         122.418680
MOD_NUMBER              -0.159215
COHORT_YEAR             -0.956432
AR_BALANCE               4.443662
ENROLLMENT_GPA           1.701210
CREDITS_ATTEMPTED        3.239262
CREDITS_EARNED           3.430428
CREDITS_REQUIRED        -1.181961
CREDITS_LEFT            -1.177201
ENROLLMENT_COUNT         2.823956
MODS_ATTENDED_COUNT      3.241725
HS_GRADUATED_FLAG        0.000000
DISABLED_FLAG          212.049523
HISPANIC_FLAG            2.763751
VETERAN_FLAG             8.332229
IN_SCHOOL_FLAG           1.877308
dtype: float64


As you can see, there are many skewness issues. Let's begin by fixing the skewness of NUMBER_AVERAGE by using one of the three options covered in the book (not including binning). Determine which option gets the skewness closest to zero. Remove any other version.

Print out the new skewness scores. 

In [2]:
# Question 2:



As you can see, we were still not able to get the skewness within an acceptable range. Several of the other features will have the same problem. Therefore, let's trying binning these features instead. Let's start by binning the ENROLLMENT_COUNT feature. Select the number of bins that gets the skewness score closest to zero. Print both the line chart depicting the various skewness scores attempted as well as the actual skewness value of the new binned version (exactly like is done in the book). 

Save this new binned version of the ENROLLMENT_COUNT feature in the DataFrame and remove any other version. Also, print out the first five records of this revised DataFrame

**HINT: Copy the code from the Practice 4 Problem**

In [3]:
# Question 3: 



As you can see, there are a couple of features with skewness scores over 100. It's very unlikely that we can do anything with those. **Run the following code** to create a loop that will examine each of those skewness scores and drop any column that has a skewness score greater than 10. Print the skewness of the remaining columns again.

In [4]:
# Question 4:

for col in df.skew().iteritems():
  if col[1] > 10:
    df.drop(columns=[col[0]], inplace=True)
  
print(len(df.columns))
df.skew()